Prophet Forescasting as Scale

Descrevemos uma abordagem prática para a previsão 'em escala' que combina modelos configuráveis com análise de 
desempenho do analista in the loop. Propomos um modelo de regressão com parâmetros interpretáveis que podem ser ajustados intuitivamente por analistas com conhecimento de dominío de sobre séries temporais. Descrevemos as análises de desempenho para comparar e avaliar os procedimentos de previsão e sinalizar automaticamente as previsões para revisão e ajuste manual. As ferramentas que ajudam os analistas a usar sua expêriencia de forma mais eficaz permitem previsões confiáveis e práticas de séries temporais de negócios.

Explorando Features do prophet

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.github.com/facebook/prophet')

In [ ]:
df.plot()

In [ ]:
model = Prophet
model.fit(df)

In [ ]:
future = model.make_future_dataframe(periods=1000)

In [ ]:
forecast = model.predict(future)
fig1 = model.plot(forecast)

In [ ]:
fig2 = model.plot_components(forecast)

Crescimeto exponencial

In [ ]:
model = Prophet(growth='logistic', changepoint_prior_scale = 0.1, seasonlity_prior_scale = 1.0)
df['cap'] = 15
df['floor'] = 0

model.fit(df)
future = model.make_future_dataframe(periods=1000)
future['cap'] = 15
future['floor'] = 0
forecast = model.predict(future)
fig1 = model.plot(forecast)
fig2 = model.plot_components(forecast)

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='730 days', period='180 days', horizon='365 days', parallel='processes')

In [ ]:
df_cv

o metódo performance_metrics pode ser usado para calcular algumas estatíticas úteis do desempenho da previsão(yhat, yhat_lower e yhat_upper em comparação com y), como uma função da distância do corte (quão longe no futuro estava a previsão). As estatísticas calculadas são o erro quadrático médio(MSE), raiz do erro quadrático médio(RMSE), erro absoluto médio(MAE), erro percentual absoluto médio(MAPE), erro percentual absoluto mediano(MDAPE) e cobertura das estimativas yhat_lower e yhat_upper. Eles são calculados em uma janela contínua das previsões em df_cv após a classificação por horizonte(ds menos corte).

In [ ]:
from fnprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

Hyperparameter tunning

In [3]:
import datetime

In [ ]:
import itertools
import numpy as np
import pandas as pd

param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

cutoffs = pd.to_datetime(['2013-02-15', '2013-08-15', '2014-02-15'])

#General all combinations of parameters

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = [] #store the RMSEs for each params here
mapes = [] #store the MAPEs for each params here
#use cross_validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df)    #fit model with givens params
    df_cv = cross_validation(m, cutoffs, horizon='30 days', parallel='processes')
    df_p = performance_metrics(df_cv)
    rmses.append(df_p['rmse'].values[0])
    mapes.append(df_p['mape'].values[0])
#Encontrando os melhores parametros
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results['mape'] = mapes
print(tuning_results)

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
model = Prophet(**best_params)
model.fit(df)

future = model.make_future_dataframe(periods=1000)

forecast = model.predict(future)
fig1 = model.plot(forecast)